## Project Group 2
#### Gabrielle H. B. Madsen, Jakob H. Schauser, Martin Pries-Brøndberg & Monika Haubro


### Import af biblioteker
Indledningsvis importerer vi relevante biblioteker til besvarelse af opgaverne.

- Med import af *requests* biblioteket bliver det muligt for os at foretage HTTP forespørgsler. Det er i høj grad en nødvendighed i task 4, hvor vi bliver bedt om at scrape fra et websted.

- Vi har ligeledes importeret *BeautifulSoup4* biblioteket med det formål at scrape fra et website. Med dette kan vi mere præcist udtrække data fra HTML. Dette relaterer sig dermed til task 4.

- I forbindelse med task 4 har vi valgt at importere *tdqm* - en progress bar, da det kan være en langsommelig proces at hente data fra et website.

- Sidst har vi valgt at importere *pandas*, der tilbyder værktøjer til at undersøge, analysere og manipulere data. Det er derfor noget, vi anvender i både task 2, 3 og 4.

In [47]:
import requests 
from bs4 import BeautifulSoup  
from tqdm.notebook import tqdm 
import pandas as pd 

### Task 2
<i>Bemærk: Undersøgelse og opdagelse af datasættet fra Task 3 ligger i dette tilfælde sammen med Task 2 grundet overlap af opgaverne. </i>

Vi starter med at indlæse selve datasættet ved hjælp af pakken *pandas*. <br>
Herefter kalder vi head() for at få et overblik over, hvordan datasættet ser ud.

Vi observerer, at der findes NaN (= Not a Number), hvilket er med i vores overvejelser senere, når der skal fjernes irrelevante kolonner fra datasættet.

In [48]:
# url = "https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv"
# file = urllib.requt.urlopen(url)
df_fakenews = pd.read_csv('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv')
df_fakenews.head() # The first five rows of our dataset

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary
0,0,141,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,Sometimes the power of Christmas will make you...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Church Congregation Brings Gift to Waitresses ...,Ruth Harris,NaN,[''],NaN,NaN,NaN
1,1,256,beforeitsnews.com,fake,http://beforeitsnews.com/awakening-start-here/...,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,Zurich Times,NaN,[''],NaN,NaN,NaN
2,2,700,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,Never Hike Alone: A Friday the 13th Fan Film U...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Never Hike Alone - A Friday the 13th Fan Film ...,NaN,NaN,[''],Never Hike Alone: A Friday the 13th Fan Film ...,NaN,NaN
3,3,768,awm.com,unreliable,http://awm.com/elusive-alien-of-the-sea-caught...,"When a rare shark was caught, scientists were ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Elusive ‘Alien Of The Sea ‘ Caught By Scientis...,Alexander Smith,NaN,[''],NaN,NaN,NaN
4,4,791,bipartisanreport.com,clickbait,http://bipartisanreport.com/2018/01/21/trumps-...,Donald Trump has the unnerving ability to abil...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Trump’s Genius Poll Is Complete & The Results ...,Gloria Christie,NaN,[''],NaN,NaN,NaN


Vi undersøger datasættet for unikke værdier og tomme rækker, hvorved vi kan få en forståelse for sammensætningen. Det er de første skridt i rensning/strukturering af vores data. <br>

In [49]:
# We define the amount of NaN values per column, in order to check how many values are missing
id_NaN = df_fakenews['id'].isna().sum()
domain_NaN = df_fakenews['domain'].isna().sum()
type_NaN = df_fakenews['type'].isna().sum()
url_NaN = df_fakenews['url'].isna().sum()
content_NaN = df_fakenews['content'].isna().sum()
scraped_at_NaN = df_fakenews['scraped_at'].isna().sum()
inserted_at_NaN = df_fakenews['inserted_at'].isna().sum()
updated_at_NaN = df_fakenews['updated_at'].isna().sum()
title_NaN = df_fakenews['title'].isna().sum()
authors_NaN = df_fakenews['authors'].isna().sum()
keywords_NaN = df_fakenews['keywords'].isna().sum()
meta_keywords_NaN = df_fakenews['meta_keywords'].isna().sum()
meta_description_NaN = df_fakenews['meta_description'].isna().sum()
tags_NaN = df_fakenews['tags'].isna().sum()
summary_NaN = df_fakenews['summary'].isna().sum()

list_of_NaN = [id_NaN, domain_NaN, type_NaN, url_NaN, content_NaN, scraped_at_NaN, inserted_at_NaN, updated_at_NaN, title_NaN, authors_NaN, keywords_NaN, meta_keywords_NaN, meta_description_NaN, tags_NaN, summary_NaN]

print (list_of_NaN) # List containing the NaN count for every column
print (len(df_fakenews)) # Total amount of rows in order to compare the amount of missing numbers

[0, 0, 12, 0, 0, 0, 0, 0, 0, 80, 250, 0, 196, 223, 250]
250


Herunder kalder vi nunique(), som bruges til at undersøge, hvor mange unikke værdier der er i hver kolonne. <br><br>
Det bemærkes her at nogle kolonner er helt tomme eller kun har en enkelt værdi. De kolonner fjerner vi fra datasættet, da de ikke er relevante. En enkelt kolonne, scraped_at har to værdier, hvor den ene type var scraped 4 timer efter den anden type. Vi vurderede derfor, at kolonnen ikke er relevant, og den fjernes herefter.
<br><br>
I alt er kolonnerne scraped_at, inserted_at, updated_at, keywords og summary fjernet.

In [50]:
# Unique values per column
df_fakenews.nunique()

Unnamed: 0          250
id                  250
domain               29
type                 10
url                 250
content             239
scraped_at            2
inserted_at           1
updated_at            1
title               248
authors             109
keywords              0
meta_keywords        36
meta_description     51
tags                 24
summary               0
dtype: int64

In [51]:
# We see that 'scraped_at' contains 2 different values
print (df_fakenews['scraped_at'].value_counts()) # We look at what the different values are in order to check whether the column is relevant for the dataset or not

2018-01-25 16:17:44.789555    197
2018-01-25 20:13:50.426130     53
Name: scraped_at, dtype: int64


In [52]:
# We remove these five columns from the dataset
drops = ['keywords', # Contains only NaN values
        'summary', # Contains only NaN values
        'inserted_at', # Contains identical entries
        'updated_at', # Contains identical entries
        'scraped_at'] # Contains only two values, where the second entry is four hours later then the first entry


new_data = df_fakenews # We define new_data to be the dataset where we have removed the five columns above
new_data.drop(columns = drops, inplace = True) # We remove the columns


new_data.head() # The first five rows, to assure everything went as expected

,Unnamed: 0,id,domain,type,url,content,title,authors,meta_keywords,meta_description,tags
0,0,141,awm.com,unreliable,http://awm.com/church-congregation-brings-gift...,Sometimes the power of Christmas will make you...,Church Congregation Brings Gift to Waitresses ...,Ruth Harris,[''],NaN,NaN
1,1,256,beforeitsnews.com,fake,http://beforeitsnews.com/awakening-start-here/...,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,AWAKENING OF 12 STRANDS of DNA – “Reconnecting...,Zurich Times,[''],NaN,NaN
2,2,700,cnnnext.com,unreliable,http://www.cnnnext.com/video/18526/never-hike-...,Never Hike Alone: A Friday the 13th Fan Film U...,Never Hike Alone - A Friday the 13th Fan Film ...,NaN,[''],Never Hike Alone: A Friday the 13th Fan Film ...,NaN
3,3,768,awm.com,unreliable,http://awm.com/elusive-alien-of-the-sea-caught...,"When a rare shark was caught, scientists were ...",Elusive ‘Alien Of The Sea ‘ Caught By Scientis...,Alexander Smith,[''],NaN,NaN
4,4,791,bipartisanreport.com,clickbait,http://bipartisanreport.com/2018/01/21/trumps-...,Donald Trump has the unnerving ability to abil...,Trump’s Genius Poll Is Complete & The Results ...,Gloria Christie,[''],NaN,NaN


I det følgende undersøger vi de enkelte kolonner for deres datatype med det formål at sikre os, at det stemmer overens med vores forventning, og det viser sig netop, at dataen er importeret korrekt dvs. som hhv. int64 ift. id samt object ift. strings/mixed strings. Datatyperne er derfor uredigeret.
 <br>

In [53]:
new_data.dtypes # We take a look at the different datatypes

Unnamed: 0           int64
id                   int64
domain              object
type                object
url                 object
content             object
title               object
authors             object
meta_keywords       object
meta_description    object
tags                object
dtype: object

Nedenfor undersøger vi dataen nærmere. Vi ser bl.a. at hvert domæne kun har én type (fake, political, trust etc.). Det indikerer, at typerne er labeled efter domæne og ikke efter artiklernes indhold. Denne observation stemmer overens med dokumentationen for datasættet: "*Each article has been attributed the same label as the label associated with its domain*" (https://github.com/several27/FakeNewsCorpus).

Først får vi et overblik over antallet af hver label. Dernæst kan vi sammenligne det samlede antal af hvert label med antallet af labels for hvert domæne. Til sidst kigger vi på, hvor mange unikke labels, der er knyttet til det enkelte domæne. Dermed ses det tydeligt, at hvert domæne kun har en enkelt unik label.
<br>

In [54]:
new_data['type'].value_counts() # The count of the different entries in 'type'

fake          155
conspiracy     31
political      23
unreliable      6
bias            6
junksci         6
unknown         6
reliable        3
clickbait       1
hate            1
Name: type, dtype: int64

In [55]:
t = new_data.groupby(by=['domain','type'], sort="True").id.count() # We group by domain and type, in order to look at type of domains
print(t) 

domain                     type      
21stcenturywire.com        conspiracy      1
alternet.org               political       2
americanlookout.com        bias            1
anonhq.com                 unreliable      1
awarenessact.com           conspiracy      6
awm.com                    unreliable      2
barenakedislam.com         hate            1
beforeitsnews.com          fake          155
bipartisanreport.com       clickbait       1
blackagendareport.com      unreliable      1
breakpoint.org             unreliable      1
breitbart.com              political       1
canadafreepress.com        conspiracy     24
charismanews.com           bias            1
christianpost.com          reliable        3
city-journal.org           political       2
cnnnext.com                unreliable      1
collectivelyconscious.net  junksci         1
nationalreview.com         political       1
naturalnews.com            junksci         5
strategic-culture.org      unknown         4
undergroundhealth

In [56]:
new_data.groupby(['domain'])['type'].nunique().sort_values(ascending=False)

domain
21stcenturywire.com          1
alternet.org                 1
washingtonsblog.com          1
washingtonexaminer.com       1
vdare.com                    1
unz.com                      1
undergroundhealth.com        1
strategic-culture.org        1
naturalnews.com              1
nationalreview.com           1
collectivelyconscious.net    1
cnnnext.com                  1
city-journal.org             1
christianpost.com            1
charismanews.com             1
canadafreepress.com          1
breitbart.com                1
breakpoint.org               1
blackagendareport.com        1
bipartisanreport.com         1
beforeitsnews.com            1
barenakedislam.com           1
awm.com                      1
awarenessact.com             1
anonhq.com                   1
americanlookout.com          1
wallstreetonparade.com       0
willyloman.wordpress.com     0
www.newsmax.com              0
Name: type, dtype: int64

I de næste to blokke undersøger vi to konkrete domæner, hvoraf den ene har en enkelt label og den anden ingen labels. I første tilfælde bliver det netop tydeligt, at domænet faktisk kun har en type label. I det andet tilfælde kigger vi på et domæne, der ikke har en label knyttet til sig. Det kommer til udtryk ved typen NaN.

In [57]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(new_data["type"].loc[new_data["domain"] == "beforeitsnews.com"]) # We choose a domain, beforeitsnews.com, and look at the entries for 'type', to see if it they're all labelled 'fake'

1      fake
7      fake
8      fake
10     fake
13     fake
23     fake
26     fake
27     fake
29     fake
30     fake
32     fake
34     fake
37     fake
39     fake
40     fake
42     fake
44     fake
45     fake
47     fake
50     fake
56     fake
61     fake
62     fake
63     fake
65     fake
68     fake
72     fake
75     fake
76     fake
77     fake
78     fake
80     fake
81     fake
84     fake
85     fake
88     fake
90     fake
91     fake
92     fake
93     fake
96     fake
97     fake
98     fake
99     fake
100    fake
101    fake
102    fake
104    fake
106    fake
107    fake
108    fake
109    fake
110    fake
111    fake
112    fake
113    fake
115    fake
117    fake
118    fake
119    fake
120    fake
122    fake
123    fake
124    fake
125    fake
126    fake
127    fake
128    fake
129    fake
130    fake
131    fake
132    fake
134    fake
135    fake
136    fake
137    fake
138    fake
139    fake
140    fake
141    fake
142    fake
143    fake
144    fake
145 

In [58]:
print(new_data.loc[new_data["domain"] == "www.newsmax.com"]) # It seems odd that some domains have zero entries in 'type', so we print the entries associated with one of those domains.

     Unnamed: 0     id           domain type  \
238         238  38549  www.newsmax.com  NaN   
240         240  38616  www.newsmax.com  NaN   
241         241  38652  www.newsmax.com  NaN   
242         242  38736  www.newsmax.com  NaN   
247         247  39477  www.newsmax.com  NaN   
248         248  39550  www.newsmax.com  NaN   
249         249  39558  www.newsmax.com  NaN   

                                                   url  \
238  https://www.newsmax.com/newsfront/david-letter...   
240  https://www.newsmax.com/newsfront/department-o...   
241  https://www.newsmax.com/politics/tom-perez-sla...   
242  https://www.newsmax.com/politics/chuck-schumer...   
247  https://www.newsmax.com/politics/michael-hayde...   
248  https://www.newsmax.com/newsfront/antonio-saba...   
249  https://www.newsmax.com/newsfront/bill-clinton...   

                                               content  \
238  Netflix has announced that David Letterman's n...   
240  The Department of Justice has

Vi undersøger, hvordan fordelingen er mellem artikeltyperne. Vi ser blandt andet at vores sample er domineret af 'fake' artikler, og at der ikke er nogle artikler af typen "reliable" og "satire", selvom der ifølge dokumentationen findes disse typer i det originale datasæt (https://github.com/several27/FakeNewsCorpus). Man kan derfor stille sig spørgsmålet, om samplen i det hele taget er repræsentativt for det originale datasæt.

**Rensning af content variablen** <br>
Der er potentiale for at rense content variablen, så man kan undersøge hyppigheden af specifikke ord. Det kræver dog, at man strukturerer dataen, således det er muligt at skelne mellem ord - også selvom de er adskilt af f.eks. \n. Det giver først mening at lave denne rensning, hvis der er et mere specifikt mål eller behov med undersøgelsen.<br><br>
For at se content, kan dette kode anvendes: *new_data['content'].value_counts()*

**Rensning af tags variablen** <br>
Vi så tidligere, at der kun er 24 unikke "tag lister". Med andre ord er der mange artikler, som ikke har tags tilknyttet. Det er brugbar viden, når man skal lave en model. Enten skal der implementeres noget kode, der sørger for at udfylde tags for de artikler, der mangler f.eks. ved hjælp af ovenstående content variable. Alternativt skal man ikke anvende "tags" variablen eller lave en entry 'ukendt'.
<br><br>
**Rensning af meta_keywords variablen** <br>
Meta keywords er en kategori med lister af keywords. Her vil det være brugbart at rense for gentagne keywords i hhv. lower og uppercase samt fjerne tomme værdier i form af ''.

For at se meta keywords, kan dette kode anvendes: *new_data['meta_keywords'].value_counts()*

I det følgende undersøger vi, om der er forfattere, som skriver for mere end et domæne. Det bemærkes, at "Tom Rogan" skriver for to forskellige, mens resten kun for en.

In [59]:
new_data.groupby(['authors'])['domain'].nunique().sort_values(ascending=False)

authors
Tom Rogan                                                             2
A Bad Witch'S Blog                                                    1
Michael Rubin                                                         1
Peter Schweizer                                                       1
Pawan Kadu                                                            1
                                                                     ..
Fraser Institute, Because Without America, There Is No Free World.    1
Firearms Radio Network                                                1
Evan Wyatt                                                            1
European Southern Observatory                                         1
Zurich Times                                                          1
Name: domain, Length: 109, dtype: int64

In [60]:
 print(new_data["domain"].loc[new_data["authors"] == "Tom Rogan"])

86        nationalreview.com
89    washingtonexaminer.com
Name: domain, dtype: object


Herunder undersøger vi, hvor mange artikler, de forskellige forfattere har skrevet.


In [61]:
new_data['authors'].value_counts() 

John Rolls                                                        10
Gerald Sinclair                                                    6
The Daily Sheeple                                                  6
Morgan Linton                                                      6
Lisa Haven                                                         4
                                                                  ..
Pawan Kadu                                                         1
Philip Wegmann                                                     1
Kelly Cohen                                                        1
John Anthony, Because Without America, There Is No Free World.     1
Bill Hoffmann                                                      1
Name: authors, Length: 109, dtype: int64

### Task 3

**En 'type' (fake, hate etc) per domæne** <br>
I forhold til observationerne vi har set, så er der kun en 'type' per domæne. Så det forventes, at domænet har meget at skulle sige omkring klassifikationen af artiklerne, hvis domænet er kendt i forvejen. 
Såfremt at en artikel kommer fra et ukendt domæne, så kan andre variabler hjælpe med at lave den rigtige klassifikation.  <br>

**De fleste forfattere skriver kun for et domæne** <br>
I vores sample size, skriver langt de fleste forfattere kun for et enkelt domæne, mens en enkelt forfatter skriver for to. Med andre ord, kan vi forvente en stærk korrelation mellem forfattere og domæner samt artiklernes 'type'. Dette kan potentielt bruges, hvis der er ukendte domæner, som en fremtidig model skal håndtere. Hvis det er af en kendt forfatter, kan vi udnytte denne korrelation. 

**Fake er overrepræsenteret i vores sample data** <br>
Efter at kigget på det originale datasæt, kan det bl.a. konstateres, at 'typen' 'fake' er overrepræsenteret i vores sample i forhold til det originale datasæt. Endvidere ser vi antallet af 'typerne', der forekommer i det originale datasæt er forskellige, således de ikke kan vægtes og give et meningsfuldt gennemsnit. Det er relevant at overveje, hvis en model udvikles. <br>
Med andre ord er 155 ud af 250 observationer i vores sample 'fake', hvilket vil sige, hvis vi tager en tilfældig observation, vi ikke ved noget om, vil det altid være et godt bud at vurdere, at den er 'fake', idet 'fake' udgør 62% af vores sample. I kontrast udgør 'fake' 9,86% af hele datasættet (https://github.com/several27/FakeNewsCorpus).<br>

**Artikler mangler data eller mangler kategorisering** <br>
Vi så i task 2, at der mangler data for nogle artikler. F.eks. manglede artikler fra www.newsmax.com en 'type' (om artiklen er 'fake', 'hate' etc) samt en forfatter. Dette er ikke et unikt tilfælde. Det har en påvirkning på en senere model, hvor man skal tage en beslutning om at inkludere disse data eller fjerne dem. Man skal dog være opmærksom på, at den manglende data også kan have information i sig selv. <br> Der er også artikler, som har 'typen' 'unknown', hvilket indeholder samme problemstilling som mangel på data.

Første print nedenfor viser vi www.newsmax.com, der har None i 'type' og NaN i authors. Næste print er en illustration af data med tagget 'type', som er 'unknown'.

In [62]:
new_data.loc[249]

Unnamed: 0                                                        249
id                                                              39558
domain                                                www.newsmax.com
type                                                              NaN
url                 https://www.newsmax.com/newsfront/bill-clinton...
content             Former U.S. President Bill Clinton on Monday c...
title               Bill Clinton Calls for Release of Reuters Jour...
authors                                                           NaN
meta_keywords       ['bill clinton', 'myanmar', 'calls', 'release'...
meta_description    Former U.S. President Bill Clinton Calls for R...
tags                Donald Trump, Russia, Trump Administration, Gu...
Name: 249, dtype: object

In [63]:
    print(new_data.loc[new_data["type"] == "unknown"]) 

     Unnamed: 0     id                 domain     type  \
169         169  26554  undergroundhealth.com  unknown   
178         178  27373  undergroundhealth.com  unknown   
218         218  35900  strategic-culture.org  unknown   
222         222  36471  strategic-culture.org  unknown   
223         223  36475  strategic-culture.org  unknown   
237         237  38330  strategic-culture.org  unknown   

                                                   url  \
169  https://www.undergroundhealth.com/best-type-ma...   
178  https://www.undergroundhealth.com/understandin...   
218  https://www.strategic-culture.org/news/2017/01...   
222  https://www.strategic-culture.org/pview/2017/1...   
223  https://www.strategic-culture.org/news/2013/02...   
237  https://www.strategic-culture.org/pview/2017/1...   

                                               content  \
169  In our modern world, it is very common for peo...   
178  by Giovani – Liveanddare.com\n\nAre you truly ...   
218  Is Viet

### Task 4

In [64]:
# Find our letters
group_nr = 2
letters = "ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]

Efter at have inspiceret den givne hjemmeside, er det blevet tydeligt, at de urls, der forbindes med de enkelte bogstaver, ligner hinanden bortset fra sidste char, der afhænger af, hvilket bogstav, der er tale om.

Derfor har vi valgt at lave en liste, der indeholder disse urls for herefter at kunne udforske hver enkelt side yderligere.

Det har vi håndteret ved at anvende et for-loop, der itererer igennem en liste af de bogstaver, vi har fået givet ud fra vores gruppenr. Hvert bogstav placeres for enden af den fælles del af urlen, således der opstår en række unikke urls, der herefter tilføjes til listen url_list.

In [65]:
base = "https://en.wikinews.org" # For later use

# Create list with urls depending on letters
url_list = []
for l in letters:
    url_list.append(f'https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from={l}')

Vi har implementeret et for-loop, der itererer igennem listen af urls (url_list) og listen af de bogstaver, vi har fået tildelt (letters) for at søge efter artikler, der starter med et givent bogstav.

I første omgang stødte vi på den udfordring, at der i nogle tilfælde var en enkelt side med sådanne artikler, mens der i andre tilfælde var flere sider. Det løste vi ved at lave et while-loop, der sørger for at tjekke, om alle artikler med det pågældende forbogstav er fundet. Inde i while-loopet tjekker vi mere konkret, hvorvidt den sidste artikel på siden har det pågældende forbogstav. Hvis ikke, må det betyde, at der ikke er flere relevante artikler at finde. Men hvis det derimod er tilfældet, må det betyde, at der kan være en side med artikler mere. Derfor har vi lavet et stykke kode, der leder efter "next page"-knappen på siden og følger dennes link.

Så længe while-loopet kører, bliver urlen læst. Der bliver oprettet en BeautifulSoup. Vi indsnævrer søgefeltet til den del af siden, vi er interesseret i vha. find og id, og vi finder selve artiklerne vha. find_all.

Herfra har vi udtrukket titlerne på artiklerne ved brug af .text og gemt dem i en liste. Alle links har vi ligeledes gemt i en liste - udtrukket ved at anvende .get og søge efter 'href'. 

Vi har naturligvis kun været interesseret i at udtrække den relevante data, hvorfor vi har tilpasset koden flere steder f.eks. har vi tilføjet [:6] == "/wiki/" efter .get for at undgå links, der ikke linker til en artikel.

In [66]:
# Create empty lists
all_titles = []
all_links = []
all_dates = []
all_content = []
all_sources = []

# Loops through url list to retrieve titles and links
for url, l in zip(url_list,letters):
    
    # Needed for doing multiple passes
    all_found = False
    page_url = url
    
    # While there are still some articles not found
    while not all_found:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.content, "html.parser")
        container = soup.find(id = "mw-pages")
        data = container.find_all('li')
        
        # Added so it only takes titles with the correct first letter
        titles = [m.text for m in data if m.text[0].upper()==l]
        
        # Append the titles to the running list
        all_titles += titles
        
        # Add links to our list
        links = [base + link.get('href') for link in container.find_all('a') if link.get('href')[:6] == "/wiki/"]
        
        all_links += links[:len(titles)]
        
        # If the last element on the page is not part of our letter we go to the next
        if data[-1].text[0] != l:
            break
            
        # Otherwise we find the "next page" button and follows it
        buttons = soup.findAll('a', href=True)
        for b in buttons:
            try:
                if b.contents[0] == "next page":
                    page_url = base + b["href"]
            except:
                pass

Med en liste af artiklernes links har det derfra været muligt for os at lave en for-loop, der itererer igennem alle artikler, læser urlen, opretter en BeautifulSoup, indsnævrer søgeområdet på siden og herfra udtrækker den data, vi er interesseret i.

Ved anvendelse af .text og find_all har vi udtrukket alle datoer og tilføjet dem til en liste. Vi har udtrukket artiklernes indhold samt referencekilder og ligeledes tilføjet dem til lister.

De endelige lister med data, vi ender ud med er all_titles, all_links, all_dates, all_content samt all_sources.

Som tidligere nævnt kan det være tidskrævende at loope igennem ca. 2650 artikler, hvorfor vi har gjort brug af *tqdm*, så det er muligt at følge, hvor mange artikler, den har indlæst.

In [67]:
# Loops through links to retrieve data from each articles
for article in tqdm(all_links):
    response = requests.get(article)
    soup = BeautifulSoup(response.content, "html.parser")
    container = soup.find('div', class_ = "mw-parser-output")

    # Date
    date = [d.text for d in container.find_all("strong", class_ = "published")]
    all_dates.append(date)

    # Content
    content = [c.text for c in container.find_all("p")[1:]]
    all_content.append(content)

    # Sources
    sources = [s.text for s in container.find_all("span", class_ = "sourceTemplate")]
    all_sources.append(sources)

  0%|          | 0/2647 [00:00<?, ?it/s]

Vi har organiseret vores data ved at navngive kolonner og specificere, hvilken liste hver kolonne skal hente data fra. Herefter har vi benyttet *pandas* til at lave en dataframe ud fra vores data. Til sidst eksporterer vi denne dataframe til hhv. csv og excel.

Det færdige datasæt indeholder 2647 rækker (scrapet 3. marts) samt 5 kolonner. Kolonnerne omfatter titler på hver enkelt artikel, deres urls/links, dato for, hvornår hver enkelt artikel er skrevet, de enkelte artiklers indhold dvs. selve artikelteksten samt den eller de kilder, den enkelte artikel er udsprunget af.

In [69]:
# Organizing data
data = {'Title': all_titles, 'Link': all_links, 'Date': all_dates, 'Content': all_content, 'Sources': all_sources}

I forbindelse med export af data til excel, skal man være opmærksom på, at det kan være nødvendigt at installere pakken *openpyxl*.

In [71]:
# Create dataframe from data
df_politics_conflict = pd.DataFrame(data)

# Export data as csv and excel
df_politics_conflict.to_csv("wikinewsdata.csv")
df_politics_conflict.to_excel("wikinewsdata.xlsx")

**No fake/no-fake label**
<br>
Efter at have arbejdet i dybden med websitet har vi vurderet, at det ikke er et fornuftigt valg at anskue artiklerne som troværdige.

Vi scraper fra et website, hvor enhver kan oprette sig som bruger og redigere indholdet. Vi har selv testet muligheden for at redigere og oprette artikler. Det kræver ikke engang en e-mail at få lov at lægge nyt indhold op eller redigere allerede eksisterende indhold. Det vil sige som enkelt person kan man nemt oprette flere tusinde konti på kort tid. Det eneste sikkerhedstjek, man møder, er captcha.

Dertil kan redigering forekomme på alle tidspunker, hvilket vil sige, at vores datasæt kan være ændret for hver gang vi scraper.

Det betyder i bund og grund, at der nemt kan opstå fake news artikler. Der vil naturligvis være en risiko forbundet med at stole på brugergenerede artikler - som er tilfældet her. Derfor vil man skulle være yderst påpasseligt med at anvende data fra denne type sider.